In [1]:
import psycopg2
import pandas as pd

In [2]:
def create_database():
    # Connecting to the postgres database
    try:
        conn = psycopg2.connect("host=127.0.0.1 dbname = postgres user = postgres password = 12345")
    except psycopg2.Error as e:
            print("Could not make connection to the postgres database ")
            print(e)
            
    # Create cursor to the database
    try:
        cur = conn.cursor()
    except psycopg2.Error as e:
        print("Could not get cursor to the database")
        print(e)
        
    # setting autocommit so that we do not have to do it after each command
    conn.set_session(autocommit = True)
    
    # Creating new database
    try:
        cur.execute("create database accounts")
    except psycopg2.Error as e:
        print('Could not create database accounts')
        print(e)
    
    # Now closing connection to the previous postgres database
    try:
        conn.close()
    except psycopg2.Error as e:
        print("could not close connection to postgres database")
        print(e)
    
    # Creating new connection and cursor to the new database
    try:
        conn = psycopg2.connect("host=127.0.0.1 dbname = accounts user = postgres password = 12345")
    except psycopg2.Error as e:
            print("Could not make connection to the accounts database ")
            print(e)

    # Creating students table
    try:
        cur = conn.cursor()
    except psycopg2.Error as e:
        print("Could not get cursor to the database")
        print(e)

    conn.set_session(autocommit = True)

    return conn, cur

In [3]:
# creating method to drop table

def drop_table(cur):
    for query in drop_table_queries:
        cur.execute(query)

In [4]:
# creating method to create table

def create_table(cur):
    for query in create_table_queries:
        cur.execute(query)

In [23]:
# loading countries data

accounts_countries = pd.read_csv('Wealth-AccountsCountry.csv')

In [24]:
# Selecting only relevant columns

accounts_countries = accounts_countries[['Code', 'Income Group', 'Region', 'Lending category',
           'Currency Unit',
       'Vital registration complete', 'Latest agricultural census',
       'Latest industrial data', 'Latest trade data',
       'Latest water withdrawal data', '2-alpha code', 'WB-2 code','Short Name']]

In [25]:
accounts_countries.head()

,Code,Income Group,Region,Lending category,Currency Unit,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,2-alpha code,WB-2 code,Short Name
0,ALB,Upper middle income,Europe & Central Asia,IBRD,Albanian lek,Yes,2012,2013.0,2018.0,2006.0,AL,AL,Albania
1,ARG,Upper middle income,Latin America & Caribbean,IBRD,Argentine peso,Yes,2008,2002.0,2018.0,2011.0,AR,AR,Argentina
2,ARM,Upper middle income,Europe & Central Asia,IBRD,Armenian dram,Yes,2014,NaN,2018.0,2012.0,AM,AM,Armenia
3,AUS,High income,East Asia & Pacific,NaN,Australian dollar,Yes,2015-2016,2013.0,2018.0,2013.0,AU,AU,Australia
4,AUT,High income,Europe & Central Asia,NaN,Euro,Yes,2010,2014.0,2018.0,2010.0,AT,AT,Austria


In [26]:
# Loading data

accounts_data = pd.read_csv('Wealth-AccountData.csv')

In [29]:
# Changing column names

accounts_data.columns = ['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018']

In [31]:
# droping irrelant column

accounts_data.drop('Series Name', axis = 1, inplace = True)

In [32]:
accounts_data.head()

,Country Name,Country Code,Series Code,1995,1996,1997,1998,1999,2000,2001,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Albania,ALB,NW.HCA.TO,4.49E+10,4.34E+10,3.71E+10,3.88E+10,4.22E+10,4.36E+10,4.76E+10,...,6.610000e+10,6.810000e+10,6.850000e+10,7.080000e+10,7.160000e+10,7.250000e+10,7.370000e+10,7.57E+10,7.80E+10,8.12E+10
1,Albania,ALB,NW.HCA.PC,14072.79,13708.99,11794.1,12399.48,13573.26,14118.13,15564.46,...,2.258200e+04,2.336286e+04,2.359058e+04,2.439397e+04,2.473162e+04,2.509002e+04,2.559371e+04,26326.26,27160.86,28333.68
2,Albania,ALB,NW.HCA.FEMP.PC,3468.858,3355.549,2868.997,3013.391,3259.893,3424.007,3783.821,...,2.825336e+03,2.889150e+03,2.848523e+03,2.930234e+03,2.954273e+03,2.957420e+03,2.987545e+03,3072.504,3178.417,3312.927
3,Albania,ALB,NW.HCA.MEMP.PC,6507.931,6353.737,5466.692,5901.072,6492.73,6872.019,7607.771,...,1.525319e+04,1.578992e+04,1.579854e+04,1.647360e+04,1.681013e+04,1.699138e+04,1.730701e+04,17817.1,18445.94,19245.19
4,Albania,ALB,NW.HCA.FEMA.PC,4893.006,4737.857,4059.321,4191.445,4536.972,4695.071,5169.876,...,3.529144e+03,3.613606e+03,3.603694e+03,3.683810e+03,3.696744e+03,3.719608e+03,3.767628e+03,3872.142,3992.188,4161.137


In [33]:
# Loading series data

accounts_series = pd.read_csv('Wealth-AccountSeries.csv')

In [38]:
# Selecting only relevant data

accounts_series = accounts_series[['Code', 'Indicator Name', 'Topic',
       'Unit of measure', 'Periodicity', 'Reference period']]


In [39]:
accounts_series.head()

,Code,Indicator Name,Topic,Unit of measure,Periodicity,Reference period
0,NW.HCA.TO,Human capital (constant 2018 US$),Human capital,Constant 2018 US$,Annual,1995-2018
1,NW.HCA.PC,Human capital per capita (constant 2018 US$),Human capital,Constant 2018 US$,Annual,1995-2018
2,NW.HCA.FEMP.PC,"Human capital per capita, employed female (con...",Human capital,Constant 2018 US$,Annual,1995-2018
3,NW.HCA.MEMP.PC,"Human capital per capita, employed male (const...",Human capital,Constant 2018 US$,Annual,1995-2018
4,NW.HCA.FEMA.PC,"Human capital per capita, female (constant 201...",Human capital,Constant 2018 US$,Annual,1995-2018


In [43]:
# Creating database, connection and cursor

conn, cur = create_database()

Could not create database accounts
database "accounts" already exists



In [47]:
# Creating table countries

query = """CREATE TABLE IF NOT EXISTS COUNTRIES(
Code varchar,
Income_Group varchar,
Region varchar,
Lending_category varchar,
Currency_Unit varchar,
Vital_registration varchar,
Latest_agricultural_census varchar,
Latest_industrial_data float,
Latest_trade_data float,
Latest_water_withdrawal float,
alpha2_code varchar,
WB_2_code varchar,
Short_Name varchar)"""

cur.execute(query)

In [62]:
# Inserting data into countries

accounts_country_insert = """
INSERT INTO COUNTRIES(
Code ,
Income_Group ,
Region ,
Lending_category ,
Currency_Unit ,
Vital_registration ,
Latest_agricultural_census ,
Latest_industrial_data ,
Latest_trade_data ,
Latest_water_withdrawal ,
alpha2_code ,
WB_2_code ,
Short_Name
)
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

for i, row in accounts_countries.iterrows():
    cur.execute(accounts_country_insert, list(row))

In [59]:
# Creating Data table.

query = """
CREATE TABLE IF NOT EXISTS DATA(
Country_Name VARCHAR,
Country_Code VARCHAR,
Series_Code VARCHAR,
"1995" VARCHAR,
"1996" VARCHAR,
"1997" VARCHAR,
"1998" VARCHAR,
"1999" VARCHAR,
"2000" VARCHAR,
"2001" VARCHAR,
"2002" VARCHAR,
"2003" VARCHAR,
"2004" VARCHAR,
"2005" VARCHAR,
"2006" VARCHAR,
"2007" VARCHAR,
"2008" VARCHAR,
"2009" VARCHAR,
"2010" VARCHAR,
"2011" VARCHAR,
"2012" VARCHAR,
"2013" VARCHAR,
"2014" VARCHAR,
"2015" VARCHAR,
"2016" VARCHAR,
"2017" VARCHAR,
"2018" VARCHAR
)
"""

cur.execute(query)

In [67]:
# Inserting data into Data table

accounts_data_insert = """
INSERT INTO data(
Country_Name ,
Country_Code ,
Series_Code ,
"1995" ,
"1996" ,
"1997" ,
"1998" ,
"1999" ,
"2000" ,
"2001" ,
"2002" ,
"2003" ,
"2004" ,
"2005" ,
"2006" ,
"2007" ,
"2008" ,
"2009" ,
"2010" ,
"2011" ,
"2012" ,
"2013" ,
"2014" ,
"2015" ,
"2016" ,
"2017" ,
"2018" 
)
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
"""

for i, row in accounts_data.iterrows():
    cur.execute(accounts_data_insert, list(row))

In [61]:
# Creating series table

query = """
CREATE TABLE IF NOT EXISTS SERIES(
Code VARCHAR,
Indicator_Name VARCHAR,
Topic VARCHAR,
Unit_of_measure VARCHAR,
Periodicity VARCHAR,
Reference_period VARCHAR
)
"""

cur.execute(query)

In [69]:
# Inserting data into series table

accounts_series_insert = """
INSERT INTO series(
Code ,
Indicator_Name ,
Topic ,
Unit_of_measure ,
Periodicity ,
Reference_period 
)
VALUES(%s,%s,%s,%s,%s,%s)
"""

for i, row in accounts_series.iterrows():
    cur.execute(accounts_series_insert, list(row))

In [ ]:
# Bingo!